<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

<h2 align=center><font size = 4>Assignment Week 3</font></h2>

In [1]:
#import Libs
import pandas
import requests
from bs4 import BeautifulSoup

"Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe..."

In [2]:
# The code to scrape the given Wikipedia 
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

#Create table with wiki data
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

#Create a pandas DataFrame using the wikipedia data table
#The data fram will have three columns: PostalCode, Borough, and Neighborhood
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
#filter out null rows
df = df[~df['PostalCode'].isnull()] 
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


"Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."

In [3]:
# Ignore cells with a borough that is Not assigned.
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table."

In [4]:
df_uniquepd=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))
df_uniquepd.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


"If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park."

In [5]:
df_uniquepd.loc[df_uniquepd['Neighbourhood']=="Not assigned",'Neighbourhood']=df_uniquepd.loc[df_uniquepd['Neighbourhood']=="Not assigned",'Borough']

"In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe."

In [6]:
df_uniquepd.shape

(103, 2)